## Convolutional Neural Networks and Computer Vision with TensorFlow

* Mostly Convolutional Neural Networks are used to analyze Images and based on the analysis make predictions.

* For Example:- Classify wither an image is of pizza or steak and detect wither a particular object appears in the image or not.

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt